In [1]:
# Cell 1 - Tell python where to find the modules to import from

import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline if notebook runs from the repo root or from notebooks/
cwd = Path.cwd()
project_root = cwd if (cwd / "src").exists() else cwd.parent # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print("Project root: ", project_root)

Project root:  /Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [ ]:
# Cell 2 — Imports, Load & clean inputs (real paths)

from src.core import load_data
from src.cleaning.clean_matrix import clean_matrix
from src.cleaning.clean_relius_roth_basis import clean_relius_roth_basis
from src.outputs.build_correction_file import build_correction_dataframe, write_correction_file

from src.cleaning.clean_relius_demo import clean_relius_demo

from src.config import RAW_DATA_DIR, USE_SAMPLE_DATA_DEFAULT, DateFilterConfig


if USE_SAMPLE_DATA_DEFAULT:
    matrix_path = None
    relius_roth_basis_path = None
    relius_demo_path = None
    sheet_name = 0
else:
    matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"
    relius_roth_basis_path = RAW_DATA_DIR / "real_roth_basis_relius_2025.xlsx"
    relius_demo_path = RAW_DATA_DIR / "real_demo_relius_2025.xlsx"
    sheet_name = "Sheet5"

# Optional date filtering (set to None for "All")
date_filter = None
# date_filter = DateFilterConfig(date_start="2025-07-01", date_end="2025-09-30", months=["July", "Aug", 9])


# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path)
matrix_clean = clean_matrix(matrix_raw, date_filter=date_filter)

# Load  and clean Relius Roth Basis raw data
relius_roth_basis_raw = load_data.load_relius_roth_basis_excel(path=relius_roth_basis_path, sheet_name=sheet_name)
relius_roth_basis_clean = clean_relius_roth_basis(relius_roth_basis_raw)

# Load and clean Relius Demo raw data
relius_demo_raw = load_data.load_relius_demo_excel(path=relius_demo_path)
relius_demo_clean = clean_relius_demo(relius_demo_raw)


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
print("matrix_raw DataFrame:    ", matrix_raw.shape)
print("matrix_clean DataFrame:  ", matrix_clean.shape)
print("matrix_clean 'ssn' dtype: ", matrix_clean["ssn"].dtype)
print("\n")

print("relius_roth_basis_raw:   ", relius_roth_basis_raw.shape)
print("relius_roth_basis_clean: ", relius_roth_basis_clean.shape)
print("relius_roth_basis_clean 'ssn' dtype: ", relius_roth_basis_clean["ssn"].dtype)
print("\n")
print("relius_demo_raw:       ", relius_demo_raw.shape)
print("relius_demo_clean:       ", relius_demo_clean.shape)
print("relius_demo_clean 'ssn' dtype: ", relius_demo_clean["ssn"].dtype)

matrix_raw DataFrame:     (108, 15)
matrix_clean DataFrame:   (86, 21)
matrix_clean 'ssn' dtype:  string


relius_roth_basis_raw:    (103, 6)
relius_roth_basis_clean:  (103, 11)
relius_roth_basis_clean 'ssn' dtype:  string


relius_demo_raw:        (105, 6)
relius_demo_clean:        (105, 11)
relius_demo_clean 'ssn' dtype:  string


### Date filter options
Use `DateFilterConfig` to limit transactions by date range and/or months. Range and months intersect.
Set `date_filter = None` for all data. Missing/invalid dates are excluded when filters are active.

Examples:
- All data: `date_filter = None`
- Range only: `DateFilterConfig(date_start="2025-01-01", date_end="2025-01-31")`
- Months only: `DateFilterConfig(months=["July", 8])`
- Range + months: `DateFilterConfig(date_start="2025-07-01", date_end="2025-09-30", months=["July", "Aug"])`


In [3]:
# Cell 3 — Quick view of Relius Roth Basis clean data

print("relius_roth_basis_clean Head:")
relius_roth_basis_clean.head(10)


relius_roth_basis_clean Head:


,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt,ssn_valid,amount_valid,date_valid,code_1099r_valid,validation_issues
0,300005R,280663408,Valerie,House,2013,11451.41,True,True,<NA>,<NA>,[]
59,300005R,275590956,Elizabeth,Morrison,2003,10026.23,True,True,<NA>,<NA>,[]
73,300005R,933996443,Dawn,Salinas,2012,7004.87,False,True,<NA>,<NA>,[ssn_invalid]
72,300005R,563476151,Michael,Mcdowell,2010,15013.29,True,True,<NA>,<NA>,[]
71,300005R,775257507,Brandon,Clark,2020,14838.30,True,True,<NA>,<NA>,[]
70,300005R,896760779,Phillip,Andrews,2000,10125.90,True,True,<NA>,<NA>,[]
69,300005R,735856158,Patrick,Gomez,2005,16914.19,True,True,<NA>,<NA>,[]
68,300005R,845129024,Kathleen,Martin,2012,17599.74,True,True,<NA>,<NA>,[]
67,300005R,988548965,Brian,Davis,2007,15314.09,False,True,<NA>,<NA>,[ssn_invalid]
65,300005R,928682834,Emily,Alvarez,2006,10719.93,False,True,<NA>,<NA>,[ssn_invalid]


In [4]:
# Cell 4 — Check key data inside Relius Roth Basis clean data

print("'ssn' lenght and index:\n", relius_roth_basis_clean["ssn"].str.len().value_counts(dropna=False))
print("\n'ssn' duplicates?:\n", relius_roth_basis_clean.duplicated(["plan_id", "ssn"]).sum())

'ssn' lenght and index:
 ssn
9    103
Name: count, dtype: Int64

'ssn' duplicates?:
 0


In [5]:
# Cell 5 — Test normalize_ssn function controls different types of formas from raw files

import pandas as pd
from src.core.normalizers import normalize_ssn

tests = pd.Series(["040511830", 40511830.0, "40511830.0", "040-51-1830"])
tests.map(normalize_ssn)

0    040511830
1    040511830
2    040511830
3    040511830
dtype: object

In [6]:
# Cell 6 — Check key SSNs that start with '0' to validate normalize function

ssn_zero_mask = (
    relius_roth_basis_clean["ssn"]
    .astype("string")
    .str.startswith("0")
    .fillna(False)
)
print(f"SSN count that starts with '0': {relius_roth_basis_clean.loc[ssn_zero_mask].shape[0]}")
relius_roth_basis_clean[relius_roth_basis_clean["ssn"].str.startswith("0")].head(10)

SSN count that starts with '0': 0


,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt,ssn_valid,amount_valid,date_valid,code_1099r_valid,validation_issues


In [7]:
# Cell 7 — Check column(s) that are Python 'list'

list_cols = [
    col for col in relius_roth_basis_clean.columns
    if relius_roth_basis_clean[col].apply(lambda x: isinstance(x, list)).any()
]

list_cols

['validation_issues']

Notes:
- Code Crashed in Cell #6 -> `print(relius_roth_basis_clean[relius_roth_basis_clean["ssn"].str.startswith("0").fillna(False)].value_counts().sum())`
    - since there is a column that stores a List (not string, float, ints, dates) and Python `list`is not hashable.

In [8]:
# Cell 8 — Check column´s Dtypes in relius_roth_basis_clean DataFrame

relius_roth_basis_clean.dtypes

plan_id                string[python]
ssn                    string[python]
first_name             string[python]
last_name              string[python]
first_roth_tax_year             Int64
roth_basis_amt                float64
ssn_valid                     boolean
amount_valid                  boolean
date_valid                    boolean
code_1099r_valid              boolean
validation_issues              object
dtype: object

In [9]:
# Cell 9 — Validate required columns exist (pre-flight)

required_matrix_cols = {
    "plan_id","ssn","txn_date","transaction_id","participant_name","matrix_account",
    "gross_amt","fed_taxable_amt","roth_initial_contribution_year","tax_code_1","tax_code_2"
}

# The '-' operator between sets is set difference
# “Give me all items that are in required_matrix_cols but not in matrix_clean.columns.”
missing = required_matrix_cols - set(matrix_clean.columns)

# assert 'CONDITION', "error message if condition is False"
# if it's True  -> nothing happens; code continues normally.
# if it's False -> Python raises an 'AssertionError' with the provided message.
assert not missing, f"Matrix missing columns: {missing}"

required_demo_cols = {"plan_id","ssn","dob"}
missing = required_demo_cols - set(relius_demo_clean.columns)
assert not missing, f"Demo missing columns: {missing}"

required_basis_cols = {"plan_id","ssn","first_roth_tax_year","roth_basis_amt"}
missing = required_basis_cols - set(relius_roth_basis_clean.columns)
assert not missing, f"Roth basis missing columns: {missing}"

print("✓ Required columns present")

✓ Required columns present


In [10]:
# Cell 10 — Run Roth Basis Taxable Analysis Engine

from src.engines.roth_taxable_analysis import run_roth_taxable_analysis


relius_roth_basis = run_roth_taxable_analysis(
    matrix_clean,
    relius_demo_clean,
    relius_roth_basis_clean
)

print("relius_roth_basis_df:", relius_roth_basis.shape)
relius_roth_basis.head(10)

relius_roth_basis_df: (26, 23)


,transaction_id,txn_date,ssn,participant_name,matrix_account,plan_id,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,...,roth_initial_contribution_year,first_roth_tax_year,start_roth_year,roth_basis_amt,age_at_txn,suggested_taxable_amt,suggested_first_roth_tax_year,correction_reason,action,match_status
0,5870943,2024-06-21,412571510,Theresa Johnson,07E00442,300005R,B,G,H,<NA>,...,2006,<NA>,2006,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
1,7092662,2024-01-06,759875909,Brittany Ruiz,07D00442,300005R,B,G,H,<NA>,...,2011,<NA>,2011,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
2,6152947,2024-02-12,123193387,Levi Ford,07E00442,300005R,B,G,H,<NA>,...,2013,<NA>,2013,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
3,2112594,2024-04-19,677603894,Jamie Wagner,07D00442,300005R,B,G,H,<NA>,...,2015,<NA>,2015,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
4,6043012,2024-12-03,943121665,Angelica Miller,07F00442,300005R,B,G,H,<NA>,...,2010,<NA>,2010,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
5,6639999,2024-08-02,197329676,David Phillips,07F00442,300005R,B,G,H,<NA>,...,2012,<NA>,2012,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
6,4955862,2024-04-18,593462606,Benjamin Pruitt,07D00442,300005R,B,G,H,<NA>,...,2015,<NA>,2015,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
7,5676787,2024-09-09,648889896,Donna Ramos,07D00442,300005R,B,G,H,<NA>,...,2006,<NA>,2006,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
8,3131959,2024-09-19,896915572,Ashley Wells,07F00442,300005R,B,G,H,<NA>,...,2012,<NA>,2012,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction
9,2361694,2024-03-30,575368651,Stacey Daniels,07C00442,300005R,B,G,H,<NA>,...,2005,<NA>,2005,NaN,<NA>,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,UPDATE_1099\nINVESTIGATE,match_needs_correction


In [11]:
# Cell 11 — Output schema check (builder-compatible canonical fields)

required_out_cols = {
    "transaction_id","txn_date","ssn","participant_name","matrix_account",
    "tax_code_1","tax_code_2","suggested_tax_code_1","suggested_tax_code_2",
    "correction_reason","action","match_status",
    "suggested_taxable_amt","suggested_first_roth_tax_year"
}
missing = required_out_cols - set(relius_roth_basis.columns)
assert not missing, f"Engine C output missing columns: {missing}"

print("✓ Engine C output schema OK (builder-compatible)")

✓ Engine C output schema OK (builder-compatible)


In [12]:
# Cell 12 — Filter validation (Roth-only + inherited excluded)

# Roth plan check based on plan_id rules:
is_roth = relius_roth_basis["plan_id"].astype(str).str.startswith("300005") | relius_roth_basis["plan_id"].astype(str).str.endswith("R")
assert is_roth.all(), "Found non-Roth plan_id rows in Engine C output." # .all() returns True only if every value in the Series is True.
                                                                        # If at least one row is False -> .all() returns False.

print("✓ Roth-only filter passed")

✓ Roth-only filter passed


In [13]:
# Cell 13 — Join coverage diagnostics (DOB + basis availability)

import pandas as pd

print("DOB missing in Engine C output:", relius_roth_basis.get("dob", pd.Series(dtype=object)).isna().sum() if "dob" in relius_roth_basis.columns else "DOB not retained")
print("first_roth_tax_year missing:", relius_roth_basis["suggested_first_roth_tax_year"].isna().sum(), "(note: this can be NA if not needed)")

DOB missing in Engine C output: DOB not retained
first_roth_tax_year missing: 25 (note: this can be NA if not needed)


In [14]:
# Cell 14 — Validate “basis coverage” rule is actually triggering

zero_taxable = relius_roth_basis[relius_roth_basis["suggested_taxable_amt"].fillna(pd.NA).eq(0.0)]
print("Rows suggesting taxable=0:", len(zero_taxable))
zero_taxable[
    [
        "plan_id","ssn","age_at_txn","gross_amt","fed_taxable_amt",
        "roth_initial_contribution_year","first_roth_tax_year", "roth_basis_amt",
        "suggested_first_roth_tax_year","suggested_taxable_amt","correction_reason","match_status","action"
    ]
].head(25)

Rows suggesting taxable=0: 0


,plan_id,ssn,age_at_txn,gross_amt,fed_taxable_amt,roth_initial_contribution_year,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action


Notes:
- This checks that suggested_taxable_amt == 0 is being produced and why.
- Already fixed: 
    - if 'roth_basis_amt' > 'gross_amount' AND 'first_roth_tax_year' == 'roth_initial_contribution_year' -> 'no correction needed' or 'qualified_roth_distribution'
    - if ppt is older than 59 1/2 AND 'current year' >= 'roth_initial_contribution_year + '5 years' -> 'no correction needed' or 'qualified_roth_distribution'
    - if 'first_roth_tax_year' != 'roth_initial_contribution_year' -> needs_correction

In [15]:
# Cell 15 — Validate the 15% proximity flag (INVESTIGATE behavior)

investigate_df = relius_roth_basis[relius_roth_basis["action"].eq("INVESTIGATE")]
print("INVESTIGATE rows:", len(investigate_df))
investigate_df[["plan_id","ssn","gross_amt","fed_taxable_amt","correction_reason","match_status","action"]].head(25)

INVESTIGATE rows: 0


,plan_id,ssn,gross_amt,fed_taxable_amt,correction_reason,match_status,action


In [16]:
# Cell 16 - Validate columns for quick export to Excel to present to stakeholders

action_df = relius_roth_basis[relius_roth_basis["action"].notna()]
print(f"NEED ACTION rows: {len(action_df)}")
export_roth_basis_df = action_df[
    [
        "plan_id","ssn","participant_name","age_at_txn", "tax_code_1",
        "tax_code_2", "suggested_tax_code_1",
        "suggested_tax_code_2", "new_tax_code", "gross_amt","fed_taxable_amt",
        "roth_initial_contribution_year","first_roth_tax_year","roth_basis_amt",
        "suggested_first_roth_tax_year","suggested_taxable_amt","correction_reason",
        "match_status","action", "matrix_account", "transaction_id", "txn_date",
    ]
]

export_roth_basis_df.head(15)

NEED ACTION rows: 26


,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,412571510,Theresa Johnson,<NA>,B,G,H,<NA>,H,13723.03,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,5870943,2024-06-21
1,300005R,759875909,Brittany Ruiz,<NA>,B,G,H,<NA>,H,10926.13,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,7092662,2024-01-06
2,300005R,123193387,Levi Ford,<NA>,B,G,H,<NA>,H,9856.98,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,6152947,2024-02-12
3,300005R,677603894,Jamie Wagner,<NA>,B,G,H,<NA>,H,11514.23,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,2112594,2024-04-19
4,300005R,943121665,Angelica Miller,<NA>,B,G,H,<NA>,H,12938.43,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,6043012,2024-12-03
5,300005R,197329676,David Phillips,<NA>,B,G,H,<NA>,H,10750.85,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,6639999,2024-08-02
6,300005R,593462606,Benjamin Pruitt,<NA>,B,G,H,<NA>,H,12491.57,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,4955862,2024-04-18
7,300005R,648889896,Donna Ramos,<NA>,B,G,H,<NA>,H,12086.43,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,5676787,2024-09-09
8,300005R,896915572,Ashley Wells,<NA>,B,G,H,<NA>,H,12930.36,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,3131959,2024-09-19
9,300005R,575368651,Stacey Daniels,<NA>,B,G,H,<NA>,H,10945.59,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07C00442,2361694,2024-03-30


In [17]:
# Cell 17 - Validate engine behavior for tax codes 'B' and 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("B") & export_roth_basis_df["tax_code_2"].eq("G")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,412571510,Theresa Johnson,<NA>,B,G,H,<NA>,H,13723.03,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,5870943,2024-06-21
1,300005R,759875909,Brittany Ruiz,<NA>,B,G,H,<NA>,H,10926.13,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,7092662,2024-01-06
2,300005R,123193387,Levi Ford,<NA>,B,G,H,<NA>,H,9856.98,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,6152947,2024-02-12
3,300005R,677603894,Jamie Wagner,<NA>,B,G,H,<NA>,H,11514.23,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,2112594,2024-04-19
4,300005R,943121665,Angelica Miller,<NA>,B,G,H,<NA>,H,12938.43,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,6043012,2024-12-03


In [18]:
# Cell 18 - Validate engine behavior for tax codes '4' and 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("4") & export_roth_basis_df["tax_code_2"].eq("G")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [19]:
# Cell 19 - Validate engine behavior for tax codes 'B' and '4'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("B") & export_roth_basis_df["tax_code_2"].eq("4")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [20]:
# Cell 20 - Validate engine behavior for tax code '4'

export_roth_basis_df[export_roth_basis_df["tax_code_1"].eq("4")].head()

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date


In [21]:
# Cell 21 - Validate engine behavior for tax code 'G'

export_roth_basis_df[export_roth_basis_df["tax_code_2"].eq("G")].head(10)

,plan_id,ssn,participant_name,age_at_txn,tax_code_1,tax_code_2,suggested_tax_code_1,suggested_tax_code_2,new_tax_code,gross_amt,...,first_roth_tax_year,roth_basis_amt,suggested_first_roth_tax_year,suggested_taxable_amt,correction_reason,match_status,action,matrix_account,transaction_id,txn_date
0,300005R,412571510,Theresa Johnson,<NA>,B,G,H,<NA>,H,13723.03,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,5870943,2024-06-21
1,300005R,759875909,Brittany Ruiz,<NA>,B,G,H,<NA>,H,10926.13,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,7092662,2024-01-06
2,300005R,123193387,Levi Ford,<NA>,B,G,H,<NA>,H,9856.98,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07E00442,6152947,2024-02-12
3,300005R,677603894,Jamie Wagner,<NA>,B,G,H,<NA>,H,11514.23,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,2112594,2024-04-19
4,300005R,943121665,Angelica Miller,<NA>,B,G,H,<NA>,H,12938.43,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,6043012,2024-12-03
5,300005R,197329676,David Phillips,<NA>,B,G,H,<NA>,H,10750.85,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,6639999,2024-08-02
6,300005R,593462606,Benjamin Pruitt,<NA>,B,G,H,<NA>,H,12491.57,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,4955862,2024-04-18
7,300005R,648889896,Donna Ramos,<NA>,B,G,H,<NA>,H,12086.43,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07D00442,5676787,2024-09-09
8,300005R,896915572,Ashley Wells,<NA>,B,G,H,<NA>,H,12930.36,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07F00442,3131959,2024-09-19
9,300005R,575368651,Stacey Daniels,<NA>,B,G,H,<NA>,H,10945.59,...,<NA>,NaN,<NA>,<NA>,- roth_rollover_code_fix_B_G_to_H\n- missing_f...,match_needs_correction,UPDATE_1099\nINVESTIGATE,07C00442,2361694,2024-03-30


'--------------------------------  Test Quick Export to Excel File --------------------------------'

In [22]:
# Cell 22 — Use quick report export for manual DataFrame output to Excel for stakeholders

from src.outputs.export_utils import write_df_excel

path = write_df_excel(export_roth_basis_df, filename_prefix="export_roth_distribs", engine="roth_taxable")

print(f"Export was successful!\nFile path: {path}")

Export was successful!
File path: /Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline/reports/outputs/roth_taxable/export_roth_distribs_20260104_210346.xlsx
